In [1]:
from langgraph.graph import StateGraph, END
import random
from typing import Dict, List, TypedDict

/Users/darkgengar/Documents/python_projects/Python-AI-Agent/myagent/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
class AgentState(TypedDict):
    name: str
    number: List[int]
    counter: int
    

In [5]:
    def greeting_node(state: AgentState) -> AgentState:
        """"greeting node to person"""
        state["name"] = f"Hi there, {state["name"]}"
        state["counter"] = 0

        return state

    def random_node(state: AgentState) -> AgentState:
        """generate a number"""
        state["number"].append(random.randint(0,10))
        state["counter"] +=1
    
        return state

    def should_continue(state: AgentState) -> AgentState:
        """func to decide what to do nxt"""
        if state["counter"] < 5:
            print("Entering loop", state["counter"])
            return "loop" # continue loop
        else:
            return "exit"
            

In [ ]:
    graph = StateGraph(AgentState)

    graph.add_node("greeting", greeting_node)
    graph.add_node("random", random_node)
    graph.add_edge("greeting", "random")

    graph.add_conditional_edges(
        "random", # src code
        should_continue, # route func
        {
            "loop"
        }
    )